In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import re
#to access our implemented ML functions
%load_ext autoreload
%autoreload 2


In [2]:
from implementations import *

In [22]:
Energy_range = [0.1, 0.2, 0.3, 0.5, 1.0, 2.0, 3.0, 5.0, 10.0, 20.0]
data_path = data_file(20.0)

In [23]:
df = build_df(data_path)

In [24]:
df[-17:-1]

,X(mm),Y(mm),Z(mm),DX,DY,DZ,KinE(MeV),dE(MeV),StepLeng,TrackLeng,NextVolume,ProcName
*,,,,,,,,,,,,
:,73.9,82.4,112,0.982,-0.0889,0.165,0.00802,gamma,NaN,NaN,NaN,NaN
:-----------------------------------------------------------------,EndOf2ndaries,Info,---------------,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,75.8,82,112,0.965,0.0985,-0.242,2.2,0.404,1.99,81.2,phantom,eIoni
:-----,List,of,2ndaries,-,#SpawnInStep=,1(Rest=,"0,Along=","0,Post=","1),",#SpawnTotal=,14,---------------
:,75.8,82,112,0.772,-0.155,0.616,0.997,e-,NaN,NaN,NaN,NaN
:-----------------------------------------------------------------,EndOf2ndaries,Info,---------------,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,78.2,82.3,112,0.758,-0.0281,-0.652,1.84,0.357,2.59,83.8,phantom,eIoni
25,79.7,82.2,111,0.474,-0.0403,-0.879,1.56,0.283,2.14,86,phantom,eIoni
26,80.5,82.1,109,0.568,-0.292,-0.77,1.28,0.284,1.79,87.7,phantom,eIoni


In [25]:
df[:10]

,X(mm),Y(mm),Z(mm),DX,DY,DZ,KinE(MeV),dE(MeV),StepLeng,TrackLeng,NextVolume,ProcName
*,,,,,,,,,,,,
Step#,X(mm),Y(mm),Z(mm),DX,DY,DZ,KinE(MeV),dE(MeV),StepLeng,TrackLeng,NextVolume,ProcName
0,0,100,100,0,0,0,20,0,0,0,phantom,msc
1,4.12,99.8,100,0.987,-0.16,-0.0122,18.4,0.563,4.12,4.12,phantom,eIoni
:-----,List,of,2ndaries,-,#SpawnInStep=,1(Rest=,"0,Along=","0,Post=","1),",#SpawnTotal=,1,---------------
:,4.12,99.8,100,0.802,0.592,0.0804,1.07,e-,NaN,NaN,NaN,NaN
:-----------------------------------------------------------------,EndOf2ndaries,Info,---------------,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8.26,99.1,100,0.974,-0.227,-0.0181,17.6,0.621,4.22,8.34,phantom,eIoni
:-----,List,of,2ndaries,-,#SpawnInStep=,1(Rest=,"0,Along=","0,Post=","1),",#SpawnTotal=,2,---------------
:,8.26,99.1,100,0.506,0.811,-0.293,0.119,e-,NaN,NaN,NaN,NaN


## Building of clean dataframe

In [87]:
df_clean = clean_df(df)

In [88]:
df_clean.head()

,index,X(mm),Y(mm),Z(mm),DX,DY,DZ,KinE(MeV),dE(MeV),StepLeng,X(mm)_s,Y(mm)_s,Z(mm)_s,DX_s,DY_s,DZ_s,Kin(MeV)_s,name_s
0,0,0.00,100.0,100.0,0.000,0.000,0.00000,20.0,0.000,0.00,0.00,0.0,0.0,0.000,0.000,0.0000,0.000,0
1,1,4.12,99.8,100.0,0.987,-0.160,-0.01220,18.4,0.563,4.12,4.12,99.8,100.0,0.802,0.592,0.0804,1.070,e-
2,2,8.26,99.1,100.0,0.974,-0.227,-0.01810,17.6,0.621,4.22,8.26,99.1,100.0,0.506,0.811,-0.2930,0.119,e-
3,3,12.90,98.2,100.0,0.983,-0.184,0.00546,16.8,0.779,4.78,0.00,0.0,0.0,0.000,0.000,0.0000,0.000,0
4,4,14.60,97.8,100.0,0.937,-0.331,0.10700,11.6,0.311,1.69,14.60,97.8,100.0,0.987,0.133,-0.0869,4.890,e-


In [97]:
pd.options.mode.chained_assignment = None
df_build = df_clean

df_build['last'] = ((df_build['KinE(MeV)'] == 0) & (df_build['index'] != 0))

df_build['cos_psi'] = 0

for row in range(1, len(df_build)):
    if(not df_build['last'][row]):
        df_build['cos_psi'] = df_build['DX'][row]*df_build['DX'][row+1] + \
                              df_build['DY'][row]*df_build['DY'][row+1] + \
                              df_build['DZ'][row]*df_build['DZ'][row+1]

df_input = df_build[["X(mm)","Y(mm)","Z(mm)","DX","DY","DZ","KinE(MeV)"]]

df_output = df_build[['dE(MeV)', 'StepLeng', "X(mm)_s","Y(mm)_s","Z(mm)_s",\
            'DX_s', 'DY_s', 'DZ_s','Kin(MeV)_s', 'name_s', 'cos_psi']]

df_output['dE(MeV)'] = df_output['dE(MeV)'] + df_output['Kin(MeV)_s']

df_output.head()

,dE(MeV),StepLeng,X(mm)_s,Y(mm)_s,Z(mm)_s,DX_s,DY_s,DZ_s,Kin(MeV)_s,name_s,cos_psi
0,0.000,0.00,0.00,0.0,0.0,0.000,0.000,0.0000,0.000,0,1.000248
1,1.633,4.12,4.12,99.8,100.0,0.802,0.592,0.0804,1.070,e-,1.000248
2,0.740,4.22,8.26,99.1,100.0,0.506,0.811,-0.2930,0.119,e-,1.000248
3,0.779,4.78,0.00,0.0,0.0,0.000,0.000,0.0000,0.000,0,1.000248
4,5.201,1.69,14.60,97.8,100.0,0.987,0.133,-0.0869,4.890,e-,1.000248


### Is it always Spawn_in_step == 1 ?

In [10]:
for E in Energy_range:
    df_check = clean_df(build_df(data_file(E)))
    Spawn_in_step = df_check[df_check.index == ':-----']['DZ']
    Spawn_in_step = Spawn_in_step.str.replace('(','',regex=True).str.replace('Rest=','',regex=True).astype('int')
    print('For Energy ', E, ' we have : ',Spawn_in_step[Spawn_in_step != 1].sum(), ' interactions that produce more than 1 one particule')

For Energy  0.1  we have :  0  interactions that produce more than 1 one particule
For Energy  0.2  we have :  0  interactions that produce more than 1 one particule
For Energy  0.3  we have :  0  interactions that produce more than 1 one particule
For Energy  0.5  we have :  0  interactions that produce more than 1 one particule
For Energy  1.0  we have :  0  interactions that produce more than 1 one particule
For Energy  2.0  we have :  0  interactions that produce more than 1 one particule
For Energy  3.0  we have :  0  interactions that produce more than 1 one particule
For Energy  5.0  we have :  0  interactions that produce more than 1 one particule
For Energy  10.0  we have :  0  interactions that produce more than 1 one particule
For Energy  20.0  we have :  0  interactions that produce more than 1 one particule


In [11]:
#Try to build the input and outputs corresponding to build a ML model

In [18]:
df_clean.head()

,index,DX,DY,DZ,KinE(MeV),dE(MeV),StepLeng,ProcName,DX_s,DY_s,DZ_s,Kin(MeV)_s,name_s
0,0,0,0,0,0.1,0,0,msc,NaN,NaN,NaN,NaN,NaN
1,1,0.995,-0.0748,0.0695,0.098,0.00196,0.048,msc,NaN,NaN,NaN,NaN,NaN
2,2,0.975,0.196,0.101,0.0969,0.00111,0.0473,msc,NaN,NaN,NaN,NaN,NaN
3,3,0.955,0.0481,0.293,0.0944,0.00257,0.0499,msc,NaN,NaN,NaN,NaN,NaN
4,4,0.917,-0.171,0.361,0.0927,0.00163,0.0482,msc,NaN,NaN,NaN,NaN,NaN


In [19]:
np_data = df_clean.to_numpy()

In [21]:
np_data[0:5, 1]

array(['0', '0.995', '0.975', '0.955', '0.917'], dtype=object)

In [16]:
np_data.shape[0]

397557

In [49]:
data_int = []
for idx, data_x in enumerate(np_data[0:20, 0:7]) :
    print(data_x.astype(float))
np_data[0:20, 8]

[0.  0.  0.  0.  0.1 0.  0. ]
[ 1.       0.995   -0.0748   0.0695   0.098    0.00196  0.048  ]
[2.00e+00 9.75e-01 1.96e-01 1.01e-01 9.69e-02 1.11e-03 4.73e-02]
[3.00e+00 9.55e-01 4.81e-02 2.93e-01 9.44e-02 2.57e-03 4.99e-02]
[ 4.00e+00  9.17e-01 -1.71e-01  3.61e-01  9.27e-02  1.63e-03  4.82e-02]
[ 5.00e+00  8.76e-01 -1.13e-01  4.68e-01  9.11e-02  1.66e-03  4.29e-02]
[ 6.00e+00  8.15e-01 -1.46e-01  5.61e-01  8.88e-02  2.27e-03  5.16e-02]
[ 7.00e+00  5.12e-01 -3.24e-01  7.95e-01  8.74e-02  1.37e-03  4.62e-02]
[ 8.00e+00  7.23e-01 -6.19e-01  3.08e-01  8.66e-02  8.12e-04  4.41e-02]
[ 9.00e+00  7.17e-01 -5.46e-01  4.32e-01  8.49e-02  1.72e-03  4.40e-02]
[ 1.00e+01  5.94e-01 -5.62e-01  5.76e-01  8.29e-02  2.03e-03  4.38e-02]
[ 1.10e+01  6.77e-01 -5.31e-01  5.09e-01  8.12e-02  1.66e-03  4.26e-02]
[ 1.20e+01  8.81e-01 -4.26e-01  2.07e-01  7.85e-02  2.75e-03  4.49e-02]
[ 1.30e+01  6.57e-01 -6.88e-01  3.09e-01  7.67e-02  1.78e-03  4.17e-02]
[ 1.40e+01  6.00e-01 -7.44e-01  2.96e-01  7.35e-02  3.1

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan], dtype=object)

In [108]:
X, y = creation_array(df_clean)

In [109]:
type(X)

numpy.ndarray

In [118]:
X[0:5]

array([[0.       , 0.1      , 0.       , 0.       , 1.       ],
       [0.       , 0.098    , 0.00196  , 0.048    , 1.       ],
       [0.9624837, 0.0969   , 0.00111  , 0.0473   , 1.       ],
       [0.9701456, 0.0944   , 0.00257  , 0.0499   , 1.       ],
       [0.9732829, 0.0927   , 0.00163  , 0.0482   , 1.       ]])

In [122]:
y[:,1].shape

(377557,)

In [113]:
y[0:20,3]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1.])

In [114]:
print((y[:,3]==1).sum())

21406


In [ ]:
#Test d'implémenter une méthode de ML sur le premier fichier

In [123]:
w_, loss = least_squares(y[:,1], X)

In [130]:

w_dist, loss_dist = least_squares(y[:,0], X)
print("Loss for the distance : {loss} \n".format(loss = loss_dist))


w_E, loss_E = least_squares(y[:,1], X)
print("Loss for the delta_E : {loss} \n".format(loss = loss_E))

w_cos, loss_cos = least_squares(y[:,2], X)
print("Loss for the cos_teta : {loss} \n".format(loss = loss_cos))

w_Q, loss_Q = least_squares(y[:,3], X)
print("Loss for the Q : {loss} \n".format(loss = loss_Q))

Loss for the distance : 0.01800725770361086 

Loss for the delta_E : 3.244195991788376e-27 

Loss for the cos_teta : 0.05344448158955016 

Loss for the Q : 0.02568517364891977 

